#**Step 1 - Write your database logic 💾**#
##Our application will store the name of the reviewer, their rating of gradio on a scale of 1 to 5, as well as any comments they want to share about the library. Let's write some code that creates a database table to store this data. We'll also write some functions to insert a review into that table and fetch the latest 10 reviews.

##We're going to use the sqlite3 library to connect to our sqlite database but gradio will work with any library.

##The code will look like this:

In [34]:
#!pip install sqlite3 # Install the sqlite3 library if not already installed
import sqlite3

In [ ]:
# Define the database file path
DB_FILE = "./reviews.db"
# Connect to the SQLite database
db = sqlite3.connect(DB_FILE)

# Attempt to create the 'reviews' table if it doesn't exist
try:
    # Try to select all rows from the 'reviews' table
    db.execute("SELECT * FROM reviews").fetchall()
    # Close the database connection if the table exists
    db.close()
except sqlite3.OperationalError:
    # If the table doesn't exist, create it
    db.execute(
        '''
        CREATE TABLE reviews (id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                              created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,
                              name TEXT, review INTEGER, comments TEXT)
                                                                                    '''
    )
    # Commit the changes to the database
    db.commit()
    # Close the database connection
    db.close()

# Function to retrieve the latest reviews from the database
def get_latest_reviews(db: sqlite3.Connection):
    # Execute a query to get the 10 most recent reviews, ordered by id descending
    reviews = db.execute("SELECT * FROM reviews ORDER BY id DESC limit 10").fetchall()
    # Get the total number of reviews in the database
    total_reviews = db.execute("Select COUNT(id) from reviews").fetchone()[0]
    # Convert the reviews to a pandas DataFrame for easier handling
    reviews = pd.DataFrame(
        reviews,
        columns=["id", "date_created", "name", "review", "comments"]
    )
    # Return the reviews DataFrame and the total number of reviews
    return reviews, total_reviews

# Function to add a new review to the database
def add_review(name: str, review: int, comments: str):
    # Connect to the database
    db = sqlite3.connect(DB_FILE)
    # Create a cursor object to execute SQL commands
    cursor = db.cursor()
    # Insert the new review into the database
    cursor.execute("INSERT INTO reviews(name, review, comments) VALUES(?,?,?)",
     [name, review, comments])
    # Commit the changes to the database
    db.commit()
    # Retrieve the updated list of reviews and total count
    reviews, total_reviews = get_latest_reviews(db)
    # Close the database connection
    db.close()
    # Return the updated reviews and total count
    return reviews, total_reviews

##Let's also write a function to load the latest reviews when the gradio application loads:

In [ ]:
def load_data():
    db = sqlite3.connect(DB_FILE)
    reviews, total_reviews = get_latest_reviews(db)
    db.close()
    return reviews, total_reviews

#**Step 2 - Create a gradio app ⚡**#
##Now that we have our database logic defined, we can use gradio create a dynamic web page to ask our users for feedback!

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.2 MB/s

In [ ]:
import gradio as gr

# Create a Gradio Blocks interface
with gr.Blocks() as demo:
    # Create a row to organize elements horizontally
    with gr.Row():
        # Create a column for input elements
        with gr.Column():
             # Create a text input for the user's name
             name = gr.Textbox(label="Name", placeholder="What is your name?")
             # Create a radio button group for rating satisfaction
             review = gr.Radio(label="How satisfied are you with using gradio?",
                               choices=[1, 2, 3, 4, 5])
             # Create a multi-line text input for comments
             comments = gr.Textbox(
                 label="Comments",
                 lines=10,
                 placeholder="Do you have any feedback on gradio?"
             )
             # Create a submit button
             submit = gr.Button(value="Submit Feedback")
        # Create a column for output elements
        with gr.Column():
            # Create a dataframe to display the most recent 10 reviews
            data = gr.Dataframe(label="Most recently created 10 rows")
            # Create a number display for the total review count
            count = gr.Number(label="Total number of reviews")
    # Define the action when the submit button is clicked
    submit.click(add_review,
     [name, review, comments],
     [data, count])
    # Define the action when the demo is loaded
    demo.load(load_data, None, [data, count])



#**Step 3 - Synchronize with HuggingFace Datasets 🤗**#
##We could call demo.launch() after step 2 and have a fully functioning application. However, our data would be stored locally on our machine. If the sqlite file were accidentally deleted, we'd lose all of our reviews! Let's back up our data to a dataset on the HuggingFace hub.

##Create a dataset here before proceeding.

##Now at the top of our script, we'll use the huggingface hub client library to connect to our dataset and pull the latest backup.

In [ ]:
!pip install huggingface_hub
import huggingface_hub

In [ ]:
import shutil

In [ ]:
# Retrieve the Hugging Face Hub token from environment variables
TOKEN = TOKEN

# Create a Repository object for interacting with a Hugging Face dataset
repo = huggingface_hub.Repository(
    # Specify the local directory where the repository will be cloned
    local_dir="data",
    # Set the repository type to "dataset"
    repo_type="dataset",
    # Specify the name of the dataset to clone from Hugging Face Hub
    clone_from="Kilos1/my-reviews",
    # Use the authentication token for accessing the repository
    use_auth_token=TOKEN
)

# Pull the latest changes from the remote repository
repo.git_pull()

# Copy the reviews database file from the cloned repository to the local DB_FILE location
shutil.copyfile("./data/reviews.db", DB_FILE)


In [ ]:
# Retrieve the Hugging Face Hub token from environment variables
TOKEN = openai_api_key

# Create a Repository object for interacting with a Hugging Face dataset
repo = huggingface_hub.Repository(
    # Specify the local directory where the repository will be cloned
    local_dir="data",
    # Set the repository type to "dataset"
    repo_type="dataset",
    # Specify the name of the dataset to clone from Hugging Face Hub
    clone_from="Kilos1/my-reviews",
    # Use the authentication token for accessing the repository
    use_auth_token=TOKEN
)

# Pull the latest changes from the remote repository
repo.git_pull()

# Check if the file exists in the expected location
import os
if os.path.exists("./data/reviews.db"):
    # Copy the reviews database file from the cloned repository to the local DB_FILE location
    shutil.copyfile("./data/reviews.db", DB_FILE)
else:
    print("File 'reviews.db' not found in the repository. Please check the file path.")
    # If the file is not in the expected location, you may need to adjust the path
    # based on its actual location in the repository.
    # For example, if the file is in a subdirectory called 'database', you would use:
    # shutil.copyfile("./data/database/reviews.db", DB_FILE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/data is already a clone of https://huggingface.co/datasets/Kilos1/my-reviews. Make sure you pull the latest changes with `repo.git_pull()`.


File 'reviews.db' not found in the repository. Please check the file path.


##Now we will create a background task to synch our local database to the dataset hub every 60 seconds. We will use the `AdvancedPythonScheduler` to handle the scheduling. However, this is not the only task scheduling library available. Feel free to use whatever you are comfortable with.

##The function to back up our data will look like this:

In [33]:
from google.colab import userdata
TOKEN = userdata.get('token')

updating db


In [ ]:
!pip install apscheduler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 788.5 kB/s eta 0:00:00


In [ ]:
# Import the BackgroundScheduler from APScheduler library
from apscheduler.schedulers.background import BackgroundScheduler
import pandas as pd
import datetime

# Define a function to backup the database
def backup_db():
    # Copy the current database file to the data directory
    shutil.copyfile(DB_FILE, "./data/reviews.db")

    # Connect to the database
    db = sqlite3.connect(DB_FILE)

    # Fetch all reviews from the database
    reviews = db.execute("SELECT * FROM reviews").fetchall()

    # Convert the reviews to a pandas DataFrame and save as CSV
    pd.DataFrame(reviews).to_csv("./data/reviews.csv", index=False)

    # Print a message indicating the update is in progress
    print("updating db")

    # Push the updated data to the Hugging Face Hub
    repo.push_to_hub(blocking=False,
                    commit_message=f"Updating data at {datetime.datetime.now()}")

# Create a BackgroundScheduler instance
scheduler = BackgroundScheduler()

# Add a job to run the backup_db function every 60 seconds
scheduler.add_job(func=backup_db,
                  trigger="interval",
                  seconds=60)
# Start the scheduler
scheduler.start()

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://527568b4302aef5b7b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
